In [1]:
%reload_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

In [2]:
import json
import pandas as pd
from gigasmol import GigaChatSmolModel
from smolagents import CodeAgent, ToolCallingAgent, Tool, UserInputTool

### Ключи api для доступа к GigaChat, сервисам погоды и курсом валют

Заполните в файле `credentials.json` соответствующие значения токенов. 

* https://www.exchangerate-api.com — сайт для получения актуальных курсов валют (получаем `currency_api_key`)
* https://openweathermap.org/api — сайт для получения актуальной информации о погоде (получаем `weather_api_key`)

In [3]:
credentials = json.load(open('credentials.json', 'r'))

### Инициализация базовой модели

In [4]:
model = GigaChatSmolModel(
    auth_data=credentials['gigachat_authorization_key'],
    client_id=credentials['gigachat_client_id'],
    model_name="GigaChat-2-Max",
    temperature=0.1,
    top_p=0.9,
    repetition_penalty=1.1,
    max_tokens=64000
)

### Инициализация календаря и почтового клиента

Используйте примеры или попробуйте инициализировать что-то свое (используйте `Mailbox` и `Calendar`)

In [5]:
from tools.mail_tools import Mailbox
from tools.calendar_tools import Calendar

from examples.mailbox_example import MAILBOX_EXAMPLE
from examples.calendar_example import CALENDAR_EXAMPLE

In [6]:
calendar = CALENDAR_EXAMPLE
mailbox = MAILBOX_EXAMPLE

In [ ]:
calendar.list_meetings()

In [ ]:
pd.DataFrame(mailbox.list_threads_with_subjects())

In [ ]:
print(mailbox.get_thread_emails_as_string("new_interface")[0])

### Инициализация агентов

In [10]:
from tools.mail_tools import MailToolset
from tools.calendar_tools import CalendarToolset
from tools.basic_tools import CurrencyConversionTool, WeatherTool, TimeTool
from tools.utils import GigaChatFinalAnswerTool

from prompts import GIGACHAT_AGENT_SYSTEM_PROMPT

In [11]:
agent = CodeAgent(
    tools=[
        CurrencyConversionTool(credentials['currency_api_key']),
        WeatherTool(credentials['weather_api_key']),
        TimeTool()] +
        MailToolset(mailbox, model).tools +
        CalendarToolset(calendar).get_tools(),
    additional_authorized_imports=["datetime"],
    model=model
)

agent.tools['final_answer'] = GigaChatFinalAnswerTool()
agent.system_prompt = GIGACHAT_AGENT_SYSTEM_PROMPT

### Интерфейс агента

In [12]:
from ui.agent_ui import GradioUI

In [13]:
# # Можно вести диалог с агентом из jupyter notebook или через интерфейс
# agent.run('Какая погода в спб, нск и москве?', reset=False)

In [14]:
# hide_steps=True — скрывает шаги в интерфейсе, но выводит их ниже в jupyter notebook 
GradioUI(agent, mailbox, calendar, hide_steps=False).launch() 

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://671a0d328f045c5f6f.gradio.live
